In [1]:
!nvidia-smi -L

In [2]:
# Hide warning logs (see: https://stackoverflow.com/a/38645250/7900723)
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Check TensorFlow version (should be 2.4.0+)
import tensorflow as tf
print(tf.__version__)

In [3]:
# Get TensorFlow Datasets
import tensorflow_datasets as tfds

In [4]:
# Load in the data (takes about 5-6 minutes in Google Colab)
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=True, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

In [5]:
# Features of Food101 TFDS
ds_info.features

In [100]:
y = np.concatenate([y for x, y in test_data], axis=0)

In [101]:
y[:10]

In [6]:
# Get class name 
class_names = ds_info.features["label"].names
class_names[:10]

In [7]:
# Take one sample off the training data
train_one_sample = train_data.take(1)

In [8]:
# What does one sample of our training data look like?
train_one_sample

In [9]:
# Output info about our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
        """)

In [10]:
# What does an image tensor from TFDS's Food101 look like?
image

In [11]:
# What are the min and max values?
tf.reduce_min(image), tf.reduce_max(image)

In [12]:
# Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()]) # add title to image by indexing on class_names list
plt.axis(False);

In [13]:
test_data

In [14]:
y_labels = []
for image, label in test_data:  # example is (image, label)
  y_labels.append(label)

In [15]:
y_labels[0]

In [16]:
def preprocess_img(image, label, img_shape=224):
    """
    Converts image datatype from 'uint8' -> 'float32' and reshapes image to 
    [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape])
    return tf.cast(image, tf.float32), label

In [17]:
# Preprocess a single image and check the outputs 
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}..., \nShape: {image.shape}, \nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}..., \nShape: {preprocessed_img.shape}, \nDatatype: {preprocessed_img.dtype}")

In [18]:
# How does it look ?
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
plt.axis(False);

In [19]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func = preprocess_img, num_parallel_calls = tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it(load it faster)
train_data = train_data.shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)

# Map preprocessing function to test data 
test_data = test_data.map(preprocess_img, num_parallel_calls = tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [20]:
train_data, test_data

In [21]:
import datetime

In [22]:
def create_tensorboard_callback(dir_name, experiment_name):
    """
    Creates a TensorBoard callback instand to store log files.
    
    Stores log files with the filepath:
        "dir_name/experiment_name/current_datetime/"

    Args: 
        dir_name: target directory to store TensorBoard log files
        experiment_name: name of experiment directory (e.g. efficientnet_model_1)
    """
    log_dir = dir_name + "/" + experiment_name +"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir
    )
    print(f"Savings TensorBoard log files to: {log_dir}")
    return tensorboard_callback


In [23]:
# Create ModelCheckpoint callback to save model's progress 
checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                      monitor = "val_accuracy", 
                                                      save_best_only = True, 
                                                      save_weights_only = True, 
                                                      verbose = 1)

In [24]:
# Turn on mixed precision training 
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy(policy = "mixed_float16")

In [25]:
mixed_precision.global_policy()

In [26]:
from tensorflow.keras import layers 
from tensorflow.keras.layers.experimental import preprocessing 

# Create base model 
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False # freeze base model layers

# Create Functional model 
inputs = layers.Input(shape = input_shape, name = "input_layer", dtype = tf.float16)
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name = "pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype = tf.float32, name = "softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

# Compile the model 
model.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics = ["accuracy"])

In [27]:
# Check out our model 
model.summary()

In [28]:
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [29]:
# Check the layers in the base model and see what dtype policy they're using 
for layer in model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [30]:
from gc import callbacks
# Fit the model with callbacks 
history_101_food_classes_feature_extract = model.fit(train_data, 
                                                     epochs = 3, 
                                                     steps_per_epoch = len(train_data), 
                                                     validation_data = test_data, 
                                                     validation_steps = int(0.15 * len(test_data)), 
                                                     callbacks = [create_tensorboard_callback("training_logs", 
                                                                                              "efficientnetb0_101_classes_all_data_feature_extract"), 
                                                                  model_checkpoint])

In [31]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data) 
results_feature_extract_model

In [32]:
# Clone the model we created 
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

In [33]:
!ls model_checkpoints/

In [34]:
checkpoint_path

In [35]:
# Load checkpointed weights into cloned_model
cloned_model.load_weights(checkpoint_path)

In [39]:
# Compile cloned_model (with same parameters as original model)
cloned_model.compile(loss = "sparse_categorical_crossentropy", 
                    optimizer = tf.keras.optimizers.Adam(), 
                    metrics = ["accuracy"])

In [40]:
# Evaluate cloned model with loaded weights (should be same score as trained model)
results_cloned_model_with_loaded_weights = cloned_model.evaluate(test_data)

In [42]:
# Save model 
save_dir = "07_efficientnetb0_feature_extract_model_mixed_precision"
model.save(save_dir)

In [43]:
# Load model previously save above
loaded_saved_model = tf.keras.models.load_model(save_dir)

In [44]:
# Check the layers in the base model and see what dtype policy they're using 
for layer in loaded_saved_model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [45]:
# Check loaded model performance 
results_loaded_saved_model = loaded_saved_model.evaluate(test_data)
results_loaded_saved_model

In [47]:
import numpy as np 
assert np.isclose(results_feature_extract_model, results_loaded_saved_model).all()

In [48]:
loaded_saved_model.summary()

In [50]:
for layer in loaded_saved_model.layers:
    layer.trainable = True
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [51]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                 patience = 3)

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                     save_best_only = True, 
                                                     monitor = "val_loss")

In [52]:
# Creating learning rate reduction callback 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_loss", 
    factor = 0.2, 
    patience = 2, 
    verbose = 1, 
    min_lr = 1e-7
)

In [53]:
# Compile the model 
loaded_saved_model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer = tf.keras.optimizers.Adam(0.0001), 
    metrics = ["accuracy"]
)

In [55]:
history_101_food_classes_all_data_fine_tune = loaded_saved_model.fit(
    train_data, 
    epochs = 100, 
    steps_per_epoch = len(train_data), 
    validation_data = test_data, 
    validation_steps = int(0.15 * len(test_data)), 
    callbacks = [create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), 
                model_checkpoint, 
                early_stopping,
                reduce_lr]
)

In [56]:
# Save model locally
loaded_saved_model.save("07_efficientnetb0_fine_tuned_101_classes_mixed_precision")

In [57]:
# Evaluate mixed precision trained loaded model
results_loaded_saved_model_fine_tuned = loaded_saved_model.evaluate(test_data)
results_loaded_saved_model_fine_tuned

In [58]:
# Load in fine-tuned-model 
loaded_fine_tuned_model = tf.keras.models.load_model("./07_efficientnetb0_fine_tuned_101_classes_mixed_precision")

In [60]:
# Get a model summary
loaded_fine_tuned_model.summary()

In [61]:
results_downloaded_fine_tuned_model = loaded_fine_tuned_model.evaluate(test_data)
results_downloaded_fine_tuned_model

In [68]:
import numpy as np 

In [77]:
y_labels_np = np.array(y_labels)

In [78]:
y_labels_np

In [79]:
y_labels

In [80]:
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics
    
    Args:
        history: TensorFlow model History object
    """
    
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]
    
    epochs = range(len(history.history["loss"]))
    
    # Plot loss 
    plt.plot(epochs, loss, label = "training_loss")
    plt.plot(epochs, val_loss, label = "val_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    
    # Plt accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label = "training_accuracy")
    plt.plot(epochs, val_accuracy, label = "val_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();

In [89]:
def compare_history(original_history, new_history, initial_epochs = 5):
    """
    Compares two TensorFlow model history objects.
    
    Args:
        original_history: History object from original model (before new_history)
        new_history: History object from continued model training (after original history)
        initial_epochs: Number of epochs in original_history (new_history plot starts from here)
        
    """
    
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]
    
    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["loss"]
    
    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]
    
    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]
    
    # Make plots
    plt.figure(figsize = (8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label = "Training Accuracy")
    plt.plot(total_val_acc, label = "Validation Accuracy")
    plt.plot([initial_epochs-1, initial_epochs - 1], 
            plt.ylim(), label = "Start Fine Tuning")
    plt.legend(loc = "lower right")
    plt.title("Training and Validation Accuracy")
    
    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label = "Training Loss")
    plt.plot(total_val_loss, label = "Validation Loss")
    plt.plot([initial_epochs - 1, initial_epochs - 1], 
            plt.ylim(), label = "Start Fine Tuning")
    plt.legend(loc = "upper right")
    plt.title("Training and Validation Loss")
    plt.xlabel("epoch")
    plt.show()
    

In [82]:
import itertools 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.metrics import confusion_matrix

In [84]:
plot_loss_curves(history_101_food_classes_feature_extract)

In [85]:
plot_loss_curves(history_101_food_classes_all_data_fine_tune)

In [90]:
compare_history(history_101_food_classes_feature_extract, history_101_food_classes_all_data_fine_tune)

In [107]:
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.io.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

In [115]:
samosa_img = "../input/inputs/samosa.jpg"

In [116]:
import matplotlib.pyplot as plt
img = load_and_prep_image(samosa_img, scale=False) # load in target image and turn it into tensor
pred_prob = loaded_saved_model.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [None, 224, 224, 3]
pred_class = class_names[pred_prob.argmax()] # find the predicted class label
# Plot the image with appropriate annotations
plt.figure()
plt.imshow(img/255.) # imshow() requires float inputs to be normalized
plt.title(f"pred: {pred_class}, prob: {pred_prob.max():.2f}")
plt.axis(False)

In [120]:
img = "../input/inputs/burger.jpg"

In [121]:
img = load_and_prep_image(img, scale=False) # load in target image and turn it into tensor
pred_prob = loaded_saved_model.predict(tf.expand_dims(img, axis=0)) # make prediction on image with shape [None, 224, 224, 3]
pred_class = class_names[pred_prob.argmax()] # find the predicted class label
# Plot the image with appropriate annotations
plt.figure()
plt.imshow(img/255.) # imshow() requires float inputs to be normalized
plt.title(f"pred: {pred_class}, prob: {pred_prob.max():.2f}")
plt.axis(False)